In [ ]:
pip install pytelegrambotapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 210 kB 5.3 MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.7.0-py3-none-any.whl size=192826 sha256=9a6089fd4206f40615d2b2557a470ae98901201adf93ce99953d40fafdbaded4
  Stored in directory: /root/.cache/pip/wheels/ba/13/da/8abf941f7cf9f993cde6118a56a7c24e12ed791507acd8ea39
Successfully built pytelegrambotapi


In [ ]:
import telebot as tb
from telebot import types
import requests as rq
import random as rd


# Конечная точка сайта с голосами птиц для России
birds_url = "https://www.xeno-canto.org/api/2/recordings?query=cnt:Russian"
wikiEndPoint = 'https://ru.wikipedia.org/api/rest_v1/page/'
username = 'koral5819@yandex.ru'
password = 'D#-R723wLcV'

bot_token = "5621803997:AAGsNlOxN6rfxkFS_18TzcnTpmf6LWAa0mc" # токен, полученный при создании бота

class Bird:
  def __init__(self, animal_spec, birdName_ru, soundUrl):
    self.soundsUrl = [] # перечень песен птицы
    self.addSound(soundUrl)
    self.animal_spec = {} # вид животного, заданный в виде словаря с ключами {'gen':..., 'sp':...}
    self.setAnimalSpec(animal_spec) # Вид птицы латиница
    self.birdViewUrl = '' # URL фотографии птицы
    self.birdName_ru = '' # русское наименование птицы
    self.setBirdName_ru(birdName_ru)
  
  # Задает видовое название птицы в виде словаря для REST запроса
  def setAnimalSpec(self, animal_spec):
    self.animal_spec = animal_spec.copy()

  # задает русское наименование птицы
  def setBirdName_ru(self, birdName_ru):
    self.birdName_ru = birdName_ru

  # Возвращает русское наименование птицы
  def getBirdName_ru(self):
    return self.birdName_ru

  # задает URL фотографии птицы
  def setBirdView(self, birdViewUrl):
    self.birdViewUrl = birdViewUrl

  # Возвращает URL фотографии птицы
  def getBirdView(self):
    return self.birdViewUrl

  # добавляет очередную песню в список песен птицы
  def addSound(self, sndUrl):
    self.soundsUrl.append(sndUrl)
  
  # возвращает песню птицы выбранную случайным образом из имеющихся для этой птицы песен
  def getBirdsSongUrl(self):
    return self.soundsUrl[rd.randint(0, len(self.soundsUrl) - 1)]

class WikiREST_API:
  def __init__(self, wikiEndPoint):
    self.wikiUrl = wikiEndPoint # Конечная точка для wiki
    self.errGet = {} # словарь ошибок при выполнении запроса
    self.json = {} # полученные данные запроса в формате json

  # возвращает информацию в формате json для заданного словаря параметров,
  # заданных в виде словаря
  def getQueryData(self, params):
    resp = rq.get(self.wikiUrl + 'summary/' + params)
    if resp.status_code == 200:
      # Запрос выполнен успешно
      json = resp.json() # полученные json данные запроса заданной страницы
      return True
    else:
      self.errGet[params] = str(resp.status_code)
      return False

  def getWikiInfoBird(self, params):
    self.clearBirdView() # очистка фотографии птицы
    resp = rq.get(self.wikiUrl + 'summary/' + params)
    if resp.status_code == 200:
      # Запрос выполнен успешно
      json = resp.json() # полученные json данные запроса заданной страницы
      self.smallViewUrl = json['thumbnail']['source']
      return True
    else:
      self.errGetViewBird[birdName_en] = str(resp.status_code)
      return False


class BirdREST_API:
  def __init__(self, user, psw, endPoint, wikiEndPoint, qtyBirds = 5):
    self.params = {"page": 1} # настраиваемые параметры запроса для сайта птиц
    self.user = user # Пользователь для сайта птиц
    self.psw = psw # Пароль пользователя для сайта птиц
    self.url = endPoint # Конечная точка для сайта птиц
    self.page = 1 # Стартовая страница запроса для сайта птиц
    self.numPages = 0 # количество страниц ответа на запрос для сайта птиц
    self.qtyBirds = qtyBirds # количество птиц для викторины
    self.bird = {} # данные о птицах в виде словаря: birdName_en : bird_obj
    self.errGet = {} # словарь ошибок при выполнении запроса по страницам
    self.recBirds = {} # словарь записей страницы птиц
    self.wikiUrl = wikiEndPoint # Конечная точка для wiki
    self.smallViewUrl = '' # url меленькой фотографии птицы
    self.errGetViewBird = {} # Словарь ошибок выполнения запроса информации по птице
    self.birdNamesForRandom = [] # Список имен птиц для случайной выборки
  
  # возвращает № страницы, выбранной случайным образом из имеющихся в запросе
  def getRandomPage(self):
    return rd.randint(1, self.numPages)

  # возвращает информацию для заданной птицы из wiki
  def getWikiInfoBird(self, animal_specis):
    self.clearBirdView() # очистка фотографии птицы
    resp = rq.get(self.wikiUrl + 'summary/' + animal_specis)
    if resp.status_code == 200:
      # Запрос выполнен успешно
      json = resp.json() # полученные json данные запроса заданной страницы
      self.smallViewUrl = json['thumbnail']['source']
      return True
    else:
      self.errGetViewBird[birdName_en] = str(resp.status_code)
      return False

  # очищает фотографию птицы
  def clearBirdView(self):
    self.smallViewUrl = ''

  # возвращает фотографию птицы
  def getBirdView(self):
    return self.smallViewUrl

  # возвращает выборку одной страницы ответа на запрос в json формате и количество страниц ответа на запрос
  def getDataBirds(self, page = 1):
    # Получаем данные с сайта
    params = {}
    params["page"] = page
    resp = rq.get(self.url, params = self.params, auth=(self.user, self.psw), allow_redirects=True)
    if resp.status_code == 200:
      # Запрос выполнен успешно
      json = resp.json() # полученные json данные запроса заданной страницы
      self.numPages = json["numPages"] # Количество страниц в запросе
      self.recBirds.clear() # предварительная очистка словаря записей птиц страницы
      self.recBirds = json['recordings']
      return True
    else:
      self.errGet[page] = str(resp.status_code)
      return False

  # заполение списка птиц для викторины случайным образом
  def getRandomLstBirds(self):
    if self.getDataBirds(self.page) == False:
      # ошибка выполнения запроса
      return False
    # ответ получен
    while len(self.bird) <  self.qtyBirds:
      # пока не занесено требуемое количество птиц
        # выбор случайной записи страницы
      record = self.recBirds[rd.randint(0, len(self.recBirds) - 1)]
      birdName_en = record["en"] # английское название птицы
      if birdName_en not in self.bird:
        # Выбранная птица еще не занесена в список викторины
        self.birdNamesForRandom.append(birdName_en)
        self.bird[birdName_en] = Bird(birdName_en, record["file"]) # добавление в список
        if self.getWikiInfoBird(birdName_en) == True:
          self.bird[birdName_en].setBirdView(self.getBirdView)
      self.page = self.getRandomPage() # Выбор очередной страницы
      if self.getDataBirds(self.page) == False:
        # ошибка выполнения запроса
        return False
    return True
    
  # возвращает список птиц, выбранных для викторины
  def getBirdsList(self):
    return self.bird

  # возвращает список ошибок выполнения запросов страниц
  # в формате {page: 'код ошибки: ....'}
  def getErrors(self):
    return self.errGet

  # Возвращает список имен птиц для случайной выборки
  def getBirdNamesForRandom(self):
    return self.birdNamesForRandom

bot = tb.TeleBot(bot_token) # Создание экземпляра класса TeleBot

LstBird = {} # Словарь выбранных птиц в формате {Имя птицы : Объект класса Bird}
birdsNames = [] # Список имен выбранный птиц для рандомизации
tstBird # класс случайно выбранной птицы

# Регистрация функции в качестве обработчика команды /start
@bot.message_handler(commands=['start', 'button'])
def start(message):
    #birds = BirdsData(username, password,birds_url)
    #birds.getData(0)
  
  birdGetData = BirdREST_API(username, password, birds_url, wikiEndPoint)
  if birdGetData.getRandomLstBirds():
    LstBird.clear()
    LstBird = birdGetData.getBirdsList()
    birdsNames = birdGetData.getBirdNamesForRandom()
    tstBird = LstBird[birdsNames[rd.randint(0, len(birdsNames) - 1)]]
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1=types.KeyboardButton(LstBird[birdsNames[0]].getBirdName_en())
    btn2=types.KeyboardButton(LstBird[birdsNames[1]].getBirdName_en())
    btn3=types.KeyboardButton(LstBird[birdsNames[2]].getBirdName_en())
    btn4=types.KeyboardButton(LstBird[birdsNames[3]].getBirdName_en())
    btn5=types.KeyboardButton(LstBird[birdsNames[4]].getBirdName_en())
    markup.add(btn1, btn2, btn3, btn4, btn5)
    bot.send_message(message.chat.id, tstBird.getBirdsSongUrl(), reply_markup=markup)
    bot.send_message(message.chat.id, 'Прослушайте песню и угадайте по ней представителя Фауны...')
  else:
    bot.send_message(message.chat.id, 'При выборке данных зафиксирована ошибка...')

@bot.message_handler(content_types=['text'])
def button_handler(message):
    if(message.text == LstBird[birdsNames[0]].getBirdName_en()):
        bot.send_message(message.chat.id, text="Привеет.. Спасибо что читаешь статью!)")
    elif(message.text == "❓ Задать вопрос"):
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        btn1 = types.KeyboardButton("Как меня зовут?")
        btn2 = types.KeyboardButton("Что я могу?")
        back = types.KeyboardButton("Вернуться в главное меню")
        markup.add(btn1, btn2, back)
        bot.send_message(message.chat.id, text="Задай мне вопрос", reply_markup=markup)
    
    elif(message.text == "Как меня зовут?"):
        bot.send_message(message.chat.id, "У меня нет имени..")
    
    elif message.text == "Что я могу?":
        bot.send_message(message.chat.id, text="Поздороваться с читателями")
    
    elif (message.text == "Вернуться в главное меню"):
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
        button1 = types.KeyboardButton("👋 Поздороваться")
        button2 = types.KeyboardButton("❓ Задать вопрос")
        markup.add(button1, button2)
        bot.send_message(message.chat.id, text="Вы вернулись в главное меню", reply_markup=markup)
    else:
        bot.send_message(message.chat.id, text="На такую комманду я не запрограммировал..")

# Постоянная отправка запроса на сервера Telegram (none_stop - не будет останавливаться даже
# в случае получения ошибки)
bot.polling(none_stop=True)